In [1]:
import re
import os

import numpy as np
import pandas as pd

/tmp/ipykernel_2406/920666016.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
paths = os.listdir("./data/")
os.chdir("./data")
names = [path.split(".txt")[0] for path in paths]
preprocess_texts = {}

for index, path in enumerate(paths):
    with open(path, "r") as f:
        preprocess_texts[names[index]] = f.read()

In [3]:
def remove_timestamps(s):
    return [tuple([i[18:].split(": ")[0], ": ".join(
        i[18:].split(": ")[1:]
    )]) for i in s.split("\n")][1:-1]

In [4]:
def remove_urls(s):
    url_regex = '(?!mailto:)(?:(?:http|https|ftp)://)(?:\\S+(?::\\S*)?@)?(?:(?:(?:[1-9]\\d?|1\\d\\d|2[01]\\d|22[0-3])(?:\\.(?:1?\\d{1,2}|2[0-4]\\d|25[0-5])){2}(?:\\.(?:[0-9]\\d?|1\\d\\d|2[0-4]\\d|25[0-4]))|(?:(?:[a-z\\u00a1-\\uffff0-9]+-?)*[a-z\\u00a1-\\uffff0-9]+)(?:\\.(?:[a-z\\u00a1-\\uffff0-9]+-?)*[a-z\\u00a1-\\uffff0-9]+)*(?:\\.(?:[a-z\\u00a1-\\uffff]{2,})))|localhost)(?::\\d{2,5})?(?:(/|\\?|#)[^\\s]*)?'
    temp_s = []

    for index, val in enumerate(s):
        _, msg = val

        if not re.search(url_regex, msg):
            temp_s.append(s[index])

    return temp_s

In [5]:
def remove_media_omitted(s):
    media_regex = "Media\somitted"
    temp_s = []

    for index, val in enumerate(s):
        _, msg = val

        if not re.search(media_regex, msg):
            temp_s.append(s[index])

    return temp_s

In [6]:
def remove_sticker_omitted(s):
    sticker_regex = "sticker\somitted"
    temp_s = []

    for index, val in enumerate(s):
        _, msg = val

        if not re.search(sticker_regex, msg):
            temp_s.append(s[index])

    return temp_s

In [7]:
def remove_audio_omitted(s):
    audio_regex = "audio\somitted"
    temp_s = []

    for index, val in enumerate(s):
        _, msg = val

        if not re.search(audio_regex, msg):
            temp_s.append(s[index])

    return temp_s

In [8]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_emojis(s):
    temp_s = []

    for val in s:
        name, text = val

        temp_s.append((name, deEmojify(text)))
    
    return temp_s

In [14]:
master_df = pd.DataFrame()
master_df.to_csv("./master.csv")

for name, text in preprocess_texts.items():
    preprocessed_text = remove_emojis(remove_audio_omitted(remove_sticker_omitted(remove_media_omitted(remove_urls(remove_timestamps(text))))))
    pd.DataFrame(preprocessed_text, columns=['name', 'line']).to_csv("./master.csv", mode="a", index=False, header=False)